In [1]:
import pandas as pd
import numpy as np
import datetime
import imp
import scripts as scr
import os
import re
import multiprocessing
import numpy as np
import pandas as pd
from pytictoc import TicToc
from IPython.display import display
import itertools
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
# import datetime


def transform_cols (df, dict_col_types = None):
    # Расширяйте для необходимых столбцов и их явной типизации
    if dict_col_types is None:
        dict_col_types = {
        'amount_original':(float, 0.0),
        'channel_indicator_desc':(str, u'null'),
        'event_description':(str, u'null'),
        'short_date':(int, 0),
        'cdf_s_20':(str, u'null'),
        'cdf_s_126':(str, u'null'),
        'cdf_s_127':(int, 30),
        'cdf_s_129':(int, 30),
        'cdf_s_138':(str, u'null'),
        'cdf_s_130':(int, 30),
        'cdf_s_133':(int, 30),
        'cdf_s_134':(int, 30),
        'cdf_s_135':(int, 30),
        'cdf_s_140':(float, 0.0),
        'cdf_s_218':(str, u'null'),
        'cdf_s_294':(int, 0),
        'cdf_s_299':(str, u'null'),
        'data_s_65':(int, 0),
        'data_i_120':(int, 0),
        'data_i_154':(float, -150)
        }
                
    if df.shape[0] == 0:
        return df
    
    df.replace(u'null', np.nan, inplace=True)

    for i in dict_col_types:
        if i in df.columns:
            change_type, fill_value = dict_col_types[i]
            df[i] = df[i].fillna(fill_value).astype(change_type)
    
    return df

# def calc_base_features(data):
#     feat_matrix = pd.DataFrame()
#     #data = data[data.event_description.isin([u'Перевод частному лицу',u'Оплата услуг',u'Перевод между своими счетами и картами'])]
    
#     if data.shape[0] == 0:
#         return feat_matrix
    
#     # заполняем ряд пропусков

#     # кумулятивная сумма опреаций за сутки, если не заполнена, то значит это первая операций, т.е. = 0
#     data.cdf_s_140 = data.cdf_s_140.fillna(0).astype(float)/1000
#     data.data_i_120.fillna(1, inplace=True)

    
#     feat_matrix['event_id'] = data.event_id
#     feat_matrix['user_id'] = data.user_id
#     feat_matrix['custom_mark'] = data.custom_mark    
#     feat_matrix['event_time'] = data.event_time
#     feat_matrix['amount'] = data.amount_original

#     feat_matrix['data_i_120'] = data.data_i_120
#     cdf_keep_cols = ['cdf_s_127', 'cdf_s_129', 'cdf_s_130', 'cdf_s_133', 'cdf_s_134', 'cdf_s_135']
#     feat_matrix.loc[:, cdf_keep_cols] = data[cdf_keep_cols]
#     feat_matrix['data_i_120'] = data.data_i_120

#     # кумулятивная сумма операций за сутки в каналах web и МП
#     feat_matrix['cumulative_sum_total'] = data.cdf_s_140

    
#     feat_matrix['client_age'] = [x.days/360 for x in (data.event_time - data.cdf_s_19)]
  
        
#     feat_matrix['cat_new_ip'] = [1 if x == u'ДА' else 0 if x == u'НЕТ' else 2 for x in data.cdf_s_126]
#     feat_matrix['cat_new_prov'] =  [1 if x == u'ДА' else 0 if x == u'НЕТ' else 2 for x in data.cdf_s_138]
#     feat_matrix['channel_op'] =  [0 if x == u'MOBILE' else 1 if x == u'WEB' else 2 for x in data.channel_indicator_desc]
#     feat_matrix['op_type'] = [0 if x == u'Перевод частному лицу' else 1 if x==u'Оплата услуг' else 2 if x ==u'Перевод между своими счетами и картами' else 3 for x in data.event_description]


#     # бинарный флаг определяющий наличие возраста получателя
#     # (полезен для линейных моделей,  менее для деревьев с учетом следующего признака)
#     feat_matrix['recip_age'] = [1 if x == 0 else 0 for x in data.cdf_s_294]
#     # разница возорастов получателей и отправителей, если отсутствует/неприменимо, то padding 500
#     feat_matrix['age_diff'] = feat_matrix.client_age - [int(x) if x != 0 else 1000 for x in data.cdf_s_294]
    
#     feat_matrix['relative'] = [1 if x == u'ДА' else 0 for x in data.cdf_s_218] # перевод родственнику
    
#     feat_matrix['know_recip_power'] = [ x if x is not None else 0 for x in data.data_s_65] # сила связи отправителя и получателя
    

#     feat_matrix['data_i_154'] = [ x if x is not None else -150 for x in data.data_i_154]
#     feat_matrix['know_recip_card_age'] = [1 if x is not None else 0 for x in data.cdf_s_124]
    
    
#     feat_matrix['recip_card_age'] = [x.days if type(x) is not pd.tslib.NaTType else 912321 for x in (data.event_time - data.cdf_s_124)]
    
#     # feat_matrix['cat_client_region'] = [x if x.isdigit() else 912321 for x in data.cdf_s_20]
#     feat_matrix['one_region'] = (data.cdf_s_20 == data.cdf_s_299).astype(int) # сравнение регионов
    

#     #ADD NEW FEATURES
#     feat_matrix['krp_pow2'] = (feat_matrix['know_recip_power']) ** 2
#     feat_matrix['log_amount'] = np.log(feat_matrix['amount'] + 1)
#     feat_matrix['ip_isp'] = np.array([x if x.isdigit() else 912321 for x in data.cdf_s_20], dtype=float)
#     feat_matrix['amnt2chnls'] = (data["amount_original"].fillna(0).astype(float) / \
#         (data["cdf_s_136"].fillna(0).astype(float) + data["amount_original"].fillna(0).astype(float) + \
#             data["amount_original"].fillna(0) + 1))
    
#     # поставил order_cols временно, чтобы сохранить такой же порядок как в оригинальном
#     order_cols = ['event_id', 'user_id', 'custom_mark', 'event_time', 'amount',
#                   'client_age', 'cat_new_ip', 'cat_new_prov', 'channel_op', 'op_type',
#                   'recip_age', 'age_diff', 'cumulative_sum_total', 'data_i_120',
#                   'relative', 'know_recip_power', 'cdf_s_127', 'cdf_s_135', 'cdf_s_130',
#                   'cdf_s_129', 'cdf_s_134', 'data_i_154', 'cdf_s_133',
#                   'know_recip_card_age', 'recip_card_age', 'one_region', 'krp_pow2',
#                   'log_amount', 'ip_isp', 'amnt2chnls']
#     feat_mat = feat_mat[order_cols].copy()
#     return feat_matrix


def load_data(chunk_fnames, fields=None, query=None, sample='train', dict_col_types=None):
    df = pd.DataFrame({})
    if isinstance(chunk_fnames, str):
        chunk_fnames = [chunk_fnames]
        
    for filename in chunk_fnames:
        chunk_df = pd.read_feather(filename)
            
        if fields is None:
            fields = chunk_df.columns.tolist()
        
        transormed = transform_cols(chunk_df)
        
        if query:
            transormed = transormed.query(query)
 
        df = pd.concat([df, transormed[fields]], ignore_index=True)
    return df


def features_handler(chunk_names, calc_feat, query=None, chunk_size=5000):
    res_df = pd.DataFrame()
    
    for chunk_name in chunk_names:
        loaded_data = load_data(chunk_name, query=query, dict_col_types=None)
        feat_chunk = calc_feat(loaded_data)
        res_df = pd.concat([res_df, feat_chunk], ignore_index=True)

    return res_df


def cust_mark_to_class(custom_mark):
    """
    Преобразует входящее значение CUSTOM_MARK в класс
    return:
        1 - фрод
        0 - легитимная
        -1 - неизвестно
    """
    ret = -1
    if custom_mark in ['F','S']:
        ret = 1
    elif custom_mark in ['A','G', np.NaN]:
        ret = 0
    
    return ret

In [3]:
FIRST_N = 100
N_THREADS = 16
train_folder = '../../data/raw_splits/train/'
train_files = sorted([x for x in os.listdir(train_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
train_files = [os.path.join(train_folder, x) for x in train_files]
print(f'Length of train files is {len(train_files)}')
train_files[:5]

Length of train files is 51


['../../data/raw_splits/train/chunk_0.fth',
 '../../data/raw_splits/train/chunk_1.fth',
 '../../data/raw_splits/train/chunk_2.fth',
 '../../data/raw_splits/train/chunk_3.fth',
 '../../data/raw_splits/train/chunk_4.fth']

In [ ]:
%%time
data_raw = pd.read_feather(train_files[0])

CPU times: user 4.03 s, sys: 2.15 s, total: 6.18 s
Wall time: 6.18 s


---

In [ ]:
%%time
data = transform_cols(data_raw)

### Подробнее что есть что

'event_id'- уникальный id транзакции

'short_date' - дата (для удобства поиска/агрегации)

'user_id' - уникальный идентификатор клиента

'event_time' - дата и время собятия

'custom_mark'- результат разбора события

'channel_indicator_desc' - канал проведения операции (web, мобильное приложение, SMS-банк и пр.)

'event_description' - описание непосредственно события (например, вход в систему или перевод, оплата услуг)

'amount_original' - сумма в рублях

'user_agent_string_hash', 'browser_plugins_hash', 'screen_hash' - различные признаки устройства (с определенной степенью точности позволяет понять уникальность устройства в разрезе пользователя)

ip_address', 'ip_country', 'ip_region', 'ip_city', 'ip_isp' - данные, связанные с IP (регион, город и интернет-провадйер по БД гео-IP)
'hardwareid' - уникальный идентификатор устройства для канала мобильных приложений

'user_acct_number_hashed'- счет отправителя (хэшированный)
'ext_acct_number_hashed'- счет получателя (хэшированный)


'data_s_65' - результат определения связи между отправителем и получателем (чем больше, тем сильнее связь) 
'data_i_118', 'data_i_119’, 'data_i_120', 'data_i_154' - ряд признаков, которые описывают устройство, с которого проводятся операции
 


 'cdf_s_136','cdf_s_137','cdf_s_140'- кумулятивные суммы операций за сутки в web, МП, web + МП
 'cdf_s_218'- предполагаемое наличие родственной связи 
 'cdf_s_127', 'cdf_s_135', 'cdf_s_130', 'cdf_s_129', 'cdf_s_134', 'cdf_s_128', 'cdf_s_138', 'cdf_s_126' - дней с момента различных рисковых событий



 'cdf_s_19'- ДР клиента
 'cdf_s_20'- Территориальный банк клиента (региональный признак)
 'cdf_s_299'- Тер банк получателя (региональный признак)

 'cdf_s_294'- возраст получателя

 'cdf_s_123'- региональный признак получателя (более локальный по сравнению с ТБ)<br>
 'cdf_s_124'- дата выдачи карты получателя

 'cdf_s_178_hashed' - реквизит получателя

In [ ]:
#%%time
tt = TicToc()
tt.tic()

feat_matrix = pd.DataFrame()

if data.shape[0] == 0:
    raise 'shape is 0'

# заполняем ряд пропусков
data.cdf_s_140 = data.cdf_s_140.fillna(0).astype(float) / 1000
data.data_i_120.fillna(1, inplace=True)

feat_matrix['amount'] = data['amount_original']
same_columns = ['event_id', 'user_id', 'event_time', 
                'cdf_s_127', 'cdf_s_129', 'cdf_s_130', 'cdf_s_133', 'cdf_s_134', 'cdf_s_135', 'data_i_120']
for column in same_columns:
    feat_matrix.loc[:, column] = data[column]
feat_matrix['is_fraud'] = [cust_mark_to_class(x) for x in data['custom_mark']]
    
# ----------
# дополнительные фичи
# по user_id
user_id_what_suffix = pd.Series([re.sub('[0-9]', '', x) for x in data['user_id']])
for suffix in ['MBK', 'VSP', 'CRM', 'IVR', 'other']:
    if suffix == 'other':
        result = ~user_id_what_suffix.isin(['MBK', 'VSP', 'CRM', 'IVR', ''])
    else:
        result = (user_id_what_suffix == suffix).astype(int)
    feat_matrix[f'user_id_{suffix}'] = result
feat_matrix['user_id_digit_only'] = feat_matrix['user_id'].apply(lambda x: x.isdigit())
# по каналу ohe
for suffix in ['MOBILEAPI', 'WEBAPI', 'ATMAPI', 'MBK', 'other']:
    if suffix == 'other':
        result = ~data.channel_indicator_desc.isin(['MOBILEAPI', 'WEBAPI', 'ATMAPI', 'MBK'])
    else:
        result = (data.channel_indicator_desc == suffix).astype(int)
    feat_matrix[f'channel_indicator_desc_is_{suffix}'] = result
# время операции
feat_matrix['event_hour'] = [x.hour for x in feat_matrix['event_time']]
feat_matrix['event_hour_night'] = [1 if ((hour >= 23) or (hour <= 7)) else 0 for hour in feat_matrix['event_hour']]
feat_matrix['event_hour_workhour'] = [1 if ((hour >= 8) or (hour <= 17)) else 0 for hour in feat_matrix['event_hour']]
feat_matrix['event_hour_evening'] = [1 if ((hour >= 18) or (hour <= 22)) else 0 for hour in feat_matrix['event_hour']]
    
feat_matrix['event_day'] = [x.dayofweek for x in feat_matrix['event_time']]
feat_matrix['event_day_is_weekend'] = [1 if day >= 6 else 0 for day in feat_matrix['event_day']]
# ----------

tt.toc('First')

In [ ]:
# кумулятивная сумма операций за сутки в каналах web и МП, умножил на 1e15 из-за того, что там сильно маленькие числа
feat_matrix['cumulative_sum_total'] = data.cdf_s_140 * 1e15


feat_matrix['client_age'] = [x.days / 365.25 for x in (data.event_time - data.cdf_s_19)]
feat_matrix['client_age_isnull'] = feat_matrix['client_age'].isnull().astype(int)

#заменил на -1
feat_matrix['cat_new_ip'] = [1 if x == 'ДА' else 0 if x == 'НЕТ' else -1 for x in data.cdf_s_126]
feat_matrix['cat_new_prov'] =  [1 if x == 'ДА' else 0 if x == 'НЕТ' else -1 for x in data.cdf_s_138]
feat_matrix['channel_op'] =  [0 if x == 'MOBILE' else 1 if x == 'WEB' else -1 for x in data.channel_indicator_desc]
feat_matrix['op_type'] = [0 if x == 'Перевод частному лицу' else
                          1 if x == 'Оплата услуг' else
                          2 if x == 'Перевод между своими счетами и картами' else
                          3 for x in data.event_description]

tt.toc('Second')

In [ ]:
# бинарный флаг определяющий наличие возраста получателя
# (полезен для линейных моделей,  менее для деревьев с учетом следующего признака)
feat_matrix['transfer_recip_age'] = [1 if x == 0 else 0 for x in data.cdf_s_294]
# разница возрастов получателей и отправителей, если отсутствует/неприменимо, то padding 500
feat_matrix['transfer_age_diff'] = feat_matrix.client_age - [int(x) if x != 0 else 1000 for x in data.cdf_s_294]
# перевод родственнику
feat_matrix['transfer_for_relative'] = [1 if x == 'ДА' else 0 for x in data.cdf_s_218] 
# сила связи отправителя и получателя
feat_matrix['transfer_know_recip_squared'] = [ x if x is not None else 0 for x in data.data_s_65]
# 'data_i_154' - ряд признаков, которые описывают устройство, с которого проводятся операции
feat_matrix['data_i_154'] = [ x if x is not None else -150 for x in data.data_i_154]
# 'cdf_s_124'- дата выдачи карты получателя
feat_matrix['know_recip_card_age'] = ~data.cdf_s_124.isnull().astype(int)
# в cdf_s_124 подмешана дата рождения, поэтому и max
feat_matrix['recip_card_age'] = [max(x.days, 1000) if type(x) is not pd.tslib.NaTType else 1000 
                                 for x in (data.event_time - data.cdf_s_124)]

# feat_matrix['cat_client_region'] = [x if x.isdigit() else 912321 for x in data.cdf_s_20]
feat_matrix['one_region'] = (data.cdf_s_20 == data.cdf_s_299).astype(int) # сравнение регионов

# там с провайдером какая-то фигня до этого была (использовалась левая переменная)
feat_matrix['ip_isp'] = data['ip_isp'].fillna(-1000000).astype(int)
tt.toc('Third')

In [ ]:
#ADD NEW FEATURES
feat_matrix['krp_pow2'] = (feat_matrix['know_recip_power']) ** 2
feat_matrix['log_amount'] = np.log(feat_matrix['amount'] + 1)

feat_matrix['client_region_len'] = data.cdf_s_20.apply(lambda x: len(str(x)))
feat_matrix['client_region'] = np.array([x if x.isdigit() else 999999 for x in data.cdf_s_20], dtype=float)
# там какая-то фигня была, переписал по смыслу
# 'cdf_s_136','cdf_s_137','cdf_s_140'- кумулятивные суммы операций за сутки в web, МП, web + МП 
feat_matrix['amnt2chnls'] = (data["amount_original"].fillna(0) / \
    (data["cdf_s_136"].fillna(0) + \
     data["cdf_s_136"].fillna(0) + \
     data["cdf_s_140"].fillna(0) + 1))
tt.toc('Fourth')

In [ ]:
# поставил order_cols временно, чтобы сохранить такой же порядок как в оригинальном
order_cols = ['event_id', 'user_id', 'custom_mark', 'event_time', 'amount',
              'client_age', 'cat_new_ip', 'cat_new_prov', 'channel_op', 'op_type',
              'recip_age', 'age_diff', 'cumulative_sum_total', 'data_i_120',
              'relative', 'know_recip_power', 'cdf_s_127', 'cdf_s_135', 'cdf_s_130',
              'cdf_s_129', 'cdf_s_134', 'data_i_154', 'cdf_s_133',
              'know_recip_card_age', 'recip_card_age', 'one_region', 'krp_pow2',
              'log_amount', 'ip_isp', 'amnt2chnls']
feat_matrix = feat_matrix[order_cols].copy()

---

In [ ]:
type(data['ip_isp'][1])

In [ ]:
data.loc[a] == 912321]

In [ ]:
data['ip_isp'].value_counts()

In [ ]:
include_channels = ['Перевод частному лицу', 'Оплата услуг', 'Перевод между своими счетами и картами']

In [ ]:
include_channels

In [ ]:
start_date = 20171029
end_date = 20171128

In [ ]:
feat_test = features_handler(
    chunk_names=[train_files[9]],
    calc_feat = calc_base_features,
    query=("event_description in {incl}  and short_date > {start} and short_date < {end}"
           .format(incl=include_channels, start=start_date, end=end_date)))

In [ ]:
feat_test.shape

In [ ]:
feat_test.columns

In [ ]:
def get_data(chunk_name):
    #del feat_test
    feat_test = features_handler(
        chunk_names=[chunk_name],
        calc_feat = calc_base_features,
        # заменил > на >=
        query=("event_description in {incl}  and short_date >= {start} and short_date =< {end}"
               .format(incl=include_channels, start=start_date, end=end_date)))
    return feat_test

In [ ]:
%%time
# жрет вплоть более 70 гигов оперативки, осторожнее
with multiprocessing.Pool(processes=min(N_THREADS, FIRST_N)) as pool:
    results = pool.map(get_data, train_files)

In [ ]:
total_df = pd.DataFrame()
for i, df in enumerate(results):
    total_df = pd.concat([total_df, df])
    results[i] = 'bye'

In [ ]:
total_df['short_date'] = total_df.event_time.apply(lambda x: x.date())
total_df.rename(columns={"custom_mark": "label"}, inplace=True)
total_df['label'] = total_df.label.apply(lambda x: scr.cust_mark_to_class(x))

In [ ]:
total_df = total_df.query("label != -1")

In [ ]:
global_mean = np.mean(total_df['label'])

In [ ]:
n_uniques = np.apply_along_axis(lambda x: len(np.unique(x)), axis=0, arr=total_df)

In [ ]:
# total_df.to_csv("../data/coms_sep/train.csv")
total_df = pd.read_csv("../data/coms_sep/train.csv", index_col=0)

In [ ]:
def map_counter(data, field_name, target_name, alpha, global_mean=None):
    if global_mean is None:
        global_mean = data[target_name].mean()
    counters = data.groupby(field_name)[target_name].mean()
    n_counters = data.groupby(field_name)[target_name].count()
    out = ((data[field_name].map(counters) + global_mean * alpha)/ \
           (data[field_name].map(n_counters) + alpha))
    return out

In [ ]:
n_uniques

In [ ]:
total_df.columns

In [ ]:
for col in total_df.columns:
    col_name = total_df.columns[i]
    if col_name not in ['event_id', "user_id", "event_time", "label", "short_date"]:
        total_df[col] = map_counter(total_df, col, 'label',
                                    alpha=1000,
                                    global_mean=global_mean)

In [ ]:
#total_df.to_csv("../data/coms_sep/train_cntrs.csv")
#total_df = pd.read_csv("../data/coms_sep/train_cntrs.csv", index_col=0)

In [ ]:
total_df.label.value_counts()